# TM10007 Assignment template

In [17]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [4]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import sklearn.linear_model 
import numpy as np

In [5]:
# Data loading functions. Uncomment the one you want to use
from hn.load_data import load_data
#from brats.load_data import load_data
#from hn.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

if data.isnull().values.any():
    print('In the csv data file, some values are missing or NaN'), sys.exit


The number of samples: 113
The number of columns: 160


In [6]:
features = data.loc[:, data.columns != 'label'].values
labels = data.loc[:,['label']].values
labels = [item if item!='T12' else 0 for item in labels]
labels = [item if item!='T34' else 1 for item in labels]
labels = np.array(labels)
print(f'Number of high risk patients: {np.count_nonzero(labels)}') 
print(f'Number of low risk patients: {len(labels) - np.count_nonzero(labels)}')

def split_sets(features, labels):
    """
    splits the features and labels into a training set (80%) and test set (20%)
    """
    x_train, x_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=1)
    return x_train, x_test, y_train, y_test 

x_train, x_test, y_train, y_test = split_sets(features, labels) 
print(x_train.shape, y_train.shape)



Number of high risk patients: 55
Number of low risk patients: 58
(90, 159) (90,)


In [79]:
def leave_one_out_val(x,y):
    """
    Leave One Out Cross Validation using Logistic Regression as a classifier
    """

    loo = LeaveOneOut()
    loo.get_n_splits(x,y)

    LeaveOneOut() 

    prediction = [] 
    y_val_total = []

    for train_index, val_index in loo.split(x,y):
        x_train, x_val = x[train_index], x[val_index]
        y_train, y_val= y[train_index], y[val_index]
    
        lrg= sklearn.linear_model.LogisticRegression()
        lrg.fit(x_train,y_train) 
    
        lrg_predicted=lrg.predict(x_val)
        prediction.append(lrg_predicted)
        y_val_total.append(y_val)
    accuracy = accuracy_score(y_val_total, prediction)

    return accuracy

accuracy = leave_one_out_val(x_train,y_train)
print(accuracy)

0.6777777777777778


In [80]:
def cross_val(x,y):
    """
    Cross validation using a Logistic Regression classifier (5 folds)
    """

    ss = StratifiedShuffleSplit(n_splits=5, test_size=0.2)            
    ss.get_n_splits(x, y)

    performances = [] 

    for train_index, val_index in ss.split(x, y):
        x_train, x_val = x[train_index], x[val_index]
        y_train, y_val= y[train_index], y[val_index]

        lrg=sklearn.linear_model.LogisticRegression()
        lrg.fit(x_train,y_train) 
        prediction=lrg.predict(x_val)
        accuracy = accuracy_score(y_val, prediction)
        performances.append(accuracy)

    return performances

accuracy = cross_val(x_train, y_train)
print(accuracy)



[0.6111111111111112, 0.6111111111111112, 0.8333333333333334, 0.5555555555555556, 0.6111111111111112]
